<a href="https://colab.research.google.com/github/npr99/PlanningMethods/blob/master/Location_Quotient_Using_BLS_QCEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Location Quotient Example Using BLS QCEW

The Bureau of Labor Statistics provides employment and wage data by year, county, state, and nation by industry.

An overview of this data is available on the BLS website for the Quarterly Census of Employment and Wages (QCEW) data

https://www.bls.gov/cew/

The BLS provides data in "data slices" - for individual states and counties.

https://data.bls.gov/cew/doc/access/csv_data_slices.htm

The BLS provides a complete layout of the datasets - specifically for Location Quotient the Annual Average data is a good choice:

https://data.bls.gov/cew/doc/access/csv_data_slices.htm#ANNUAL_LAYOUT

The BLS provides a complete list of Areas (states and counties) - the "area codes" are based on the state and county FIPS codes:

https://data.bls.gov/cew/doc/titles/area/area_titles.htm

### Example Area Slice Files

2016 QCEW Annual averages for Georgia
http://data.bls.gov/cew/data/api/2016/a/area/13000.csv

2016 QCEW Annual averages for Chatham County, GA
http://data.bls.gov/cew/data/api/2016/a/area/13051.csv

### File Layout
https://www.bls.gov/cew/about-data/downloadable-file-layouts/annual/naics-based-annual-layout.htm

#### Details on codes for different ownership types:
https://www.bls.gov/cew/classifications/ownerships/ownership-titles.htm


Note totals for employment are the sums of all Annual average of monthly employment levels for a given year (annual_avg_emplvl) for all ownership codes.

### BLS data as time series
Chatham County QCEW Time Series Data
https://data.bls.gov/timeseries/ENU1305110010


## Step 1: Obtain BLS QCEW Data File
The Pandas read csv command is a fast way to download .csv datafiles directly to the notebook session.

In [3]:
import pandas as pd # For reading, writing and wrangling data

In [4]:
blsqcew_areatitles = pd.read_csv('https://data.bls.gov/cew/doc/titles/area/area_titles.csv')
blsqcew_areatitles.head()

,area_fips,area_title
0,US000,U.S. TOTAL
1,USCMS,U.S. Combined Statistical Areas (combined)
2,USMSA,U.S. Metropolitan Statistical Areas (combined)
3,USNMS,U.S. Nonmetropolitan Area Counties (combined)
4,01000,Alabama -- Statewide


In [5]:
# Find Area FIPS code based on county name
blsqcew_areatitles.loc[blsqcew_areatitles['area_title'] == 'Chatham County, Georgia']


,area_fips,area_title
486,13051,"Chatham County, Georgia"


In [6]:
# Save Area Title for later use
area_title_df = blsqcew_areatitles.loc[blsqcew_areatitles['area_fips'] == '13051']
area_title = area_title_df['area_title'].values[0]
area_title

'Chatham County, Georgia'

In [7]:
blsqcew = pd.read_csv('http://data.bls.gov/cew/data/api/2016/a/area/US000.csv')
blsqcew.head()

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,taxable_annual_wages,annual_contributions,annual_avg_wkly_wage,avg_annual_pay,lq_disclosure_code,lq_annual_avg_estabs,lq_annual_avg_emplvl,lq_total_annual_wages,lq_taxable_annual_wages,lq_annual_contributions,lq_annual_avg_wkly_wage,lq_avg_annual_pay,oty_disclosure_code,oty_annual_avg_estabs_chg,oty_annual_avg_estabs_pct_chg,oty_annual_avg_emplvl_chg,oty_annual_avg_emplvl_pct_chg,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
0,US000,0,10,10,0,2016,A,NaN,9716618,141870066,7607227485238,1585348464192,37906449684,1031,53621,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,193843,2.0,2378367,1.7,222279680914,3.0,48119092890,3.1,-3312645329,-8.0,13,1.3,679,1.3
1,US000,1,10,11,0,2016,A,NaN,60379,2792987,218910901499,0,0,1507,78379,NaN,1.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,-418,-0.7,36553,1.3,4183583233,1.9,0,0.0,0,0.0,9,0.6,479,0.6
2,US000,1,101,12,0,2016,A,NaN,121,49904,3922642597,0,0,1512,78603,NaN,1.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,0,0.0,1475,3.0,178110904,4.8,0,0.0,0,0.0,25,1.7,1283,1.7
3,US000,1,1011,13,0,2016,A,NaN,74,3174,201308742,0,0,1220,63418,NaN,1.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,-2,-2.6,3,0.1,-5786394,-2.8,0,0.0,0,0.0,-36,-2.9,-1896,-2.9
4,US000,1,1012,13,0,2016,A,NaN,2,5,463429,0,0,1944,101112,NaN,1.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,0,0.0,1,25.0,32162,7.5,0,0.0,0,0.0,-129,-6.2,-6705,-6.2


## Step 3: Explore Data
Look at descripitive statistics for key variabiables.

In [8]:
# Explore the Location quotient of annual average employment relative to the U.S. (Rounded to the hundredths place)
blsqcew[['annual_avg_estabs','annual_avg_emplvl','lq_annual_avg_emplvl','lq_total_annual_wages',]].describe()

,annual_avg_estabs,annual_avg_emplvl,lq_annual_avg_emplvl,lq_total_annual_wages
count,4.583000e+03,4.583000e+03,4583.000000,4583.000000
mean,2.125358e+04,3.134968e+05,0.818023,0.821732
std,2.844505e+05,3.901667e+06,0.385868,0.382780
min,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,8.000000e+00,1.055000e+02,1.000000,1.000000
50%,3.450000e+02,1.126900e+04,1.000000,1.000000
75%,3.737000e+03,7.112950e+04,1.000000,1.000000
max,9.716618e+06,1.418701e+08,1.000000,1.000000


# Look at top Location Quotients by industry

## Full list of industry codes
For a full list of industry codes use the link below:

https://www.bls.gov/cew/classifications/industry/industry-titles.htm


## More details on NAICS
North American Industry Classification System (NAICS) 

https://www.naics.com/search/

## NAICS FAQ
https://www.naics.com/frequently-asked-questions/#NAICSfaq

## Add Industry Titles
The current industry titles were updated in 2017. The list of industries appears to be backwards compatitle and includes codes for all previous years and updated codes. The BLS website has a number of tables that help identify new NAICS codes and how they map to older NAICS codes. Most of the changes appear to be at the 4, 5, and 6 digit NAICS code levels.

"For detailed information on QCEW establishment, employment, and wage levels for each industry affected by the NAICS 2017 conversion, please refer to this [QCEW 2017 revision table](https://data.bls.gov/cew/apps/bls_naics/naics2017.xls)."


In [9]:
industry_titles_df = pd.read_csv('https://www.bls.gov/cew/classifications/industry/industry-titles-csv.csv')
industry_titles_df.head()

,industry_code,industry_title
0,10,"10 Total, all industries"
1,101,101 Goods-producing
2,1011,1011 Natural resources and mining
3,1012,1012 Construction
4,1013,1013 Manufacturing


In [10]:
industry_titles_df.industry_code.describe()

count     2497
unique    2497
top        721
freq         1
Name: industry_code, dtype: object

In [11]:
blsqcew_titles = pd.merge(left = industry_titles_df,
                          right = blsqcew,
                          left_on = 'industry_code',
                          right_on = 'industry_code',
                          how = 'outer')
blsqcew_titles.head()

,industry_code,industry_title,area_fips,own_code,agglvl_code,size_code,year,qtr,disclosure_code,annual_avg_estabs,annual_avg_emplvl,total_annual_wages,taxable_annual_wages,annual_contributions,annual_avg_wkly_wage,avg_annual_pay,lq_disclosure_code,lq_annual_avg_estabs,lq_annual_avg_emplvl,lq_total_annual_wages,lq_taxable_annual_wages,lq_annual_contributions,lq_annual_avg_wkly_wage,lq_avg_annual_pay,oty_disclosure_code,oty_annual_avg_estabs_chg,oty_annual_avg_estabs_pct_chg,oty_annual_avg_emplvl_chg,oty_annual_avg_emplvl_pct_chg,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
0,10,"10 Total, all industries",US000,0.0,10.0,0.0,2016.0,A,NaN,9716618.0,141870066.0,7.607227e+12,1.585348e+12,3.790645e+10,1031.0,53621.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,193843.0,2.0,2378367.0,1.7,2.222797e+11,3.0,4.811909e+10,3.1,-3.312645e+09,-8.0,13.0,1.3,679.0,1.3
1,10,"10 Total, all industries",US000,1.0,11.0,0.0,2016.0,A,NaN,60379.0,2792987.0,2.189109e+11,0.000000e+00,0.000000e+00,1507.0,78379.0,NaN,1.0,1.0,1.0,0.0,0.0,1.0,1.0,NaN,-418.0,-0.7,36553.0,1.3,4.183583e+09,1.9,0.000000e+00,0.0,0.000000e+00,0.0,9.0,0.6,479.0,0.6
2,10,"10 Total, all industries",US000,2.0,11.0,0.0,2016.0,A,NaN,69215.0,4569606.0,2.612344e+11,9.536312e+09,2.895288e+07,1099.0,57168.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,215.0,0.3,2984.0,0.1,6.062855e+09,2.4,5.495933e+08,6.1,-2.637430e+06,-8.3,24.0,2.2,1290.0,2.3
3,10,"10 Total, all industries",US000,3.0,11.0,0.0,2016.0,A,NaN,169349.0,14002852.0,6.783048e+11,2.173551e+10,2.156006e+08,932.0,48440.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,707.0,0.4,141926.0,1.0,1.890473e+10,2.9,2.348815e+08,1.1,-9.791392e+06,-4.3,17.0,1.9,867.0,1.8
4,10,"10 Total, all industries",US000,5.0,11.0,0.0,2016.0,A,NaN,9417676.0,120504622.0,6.448777e+12,1.554077e+12,3.766190e+10,1029.0,53515.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,193340.0,2.1,2196905.0,1.9,1.931285e+11,3.1,4.733462e+10,3.1,-3.300217e+09,-8.1,12.0,1.2,639.0,1.2


In [12]:
blsqcew_titles.industry_code.describe()

count     4836
unique    2497
top         10
freq         7
Name: industry_code, dtype: object

In [13]:
import numpy as np
blsqcew_titles[['industry_code','industry_title','annual_avg_emplvl']].\
loc[blsqcew_titles['annual_avg_emplvl'].isnull()].head()

,industry_code,industry_title,annual_avg_emplvl
297,21112,NAICS 21112 Crude petroleum extraction,NaN
298,211120,NAICS 211120 Crude petroleum extraction,NaN
299,21113,NAICS 21113 Natural gas extraction,NaN
300,211130,NAICS 211130 Natural gas extraction,NaN
315,212230,"NAICS 212230 Copper, nickel, lead, and zinc mi...",NaN


In [14]:
blsqcew_titles[['industry_title','annual_avg_estabs','annual_avg_emplvl','lq_annual_avg_emplvl','lq_total_annual_wages']].sort_values(by='lq_annual_avg_emplvl', ascending=False).head(10)

,industry_title,annual_avg_estabs,annual_avg_emplvl,lq_annual_avg_emplvl,lq_total_annual_wages
0,"10 Total, all industries",9716618.0,141870066.0,1.0,1.0
3313,NAICS 54192 Photographic services,13632.0,54410.0,1.0,1.0
3186,NAICS 541511 Custom computer programming services,18.0,31.0,1.0,1.0
3187,NAICS 541511 Custom computer programming services,86.0,2064.0,1.0,1.0
3188,NAICS 541511 Custom computer programming services,28.0,555.0,1.0,1.0
3189,NAICS 541511 Custom computer programming services,112249.0,857764.0,1.0,1.0
3190,NAICS 541512 Computer systems design services,77.0,2289.0,1.0,1.0
3191,NAICS 541512 Computer systems design services,81.0,1187.0,1.0,1.0
3192,NAICS 541512 Computer systems design services,120237.0,943061.0,1.0,1.0
3193,NAICS 541513 Computer facilities management se...,47.0,136.0,1.0,1.0


Look at top industries by employement.

In [15]:
blsqcew_titles[['industry_title','annual_avg_estabs','annual_avg_emplvl','lq_annual_avg_emplvl','lq_total_annual_wages']].sort_values(by='annual_avg_emplvl', ascending=False).head(10)

,industry_title,annual_avg_estabs,annual_avg_emplvl,lq_annual_avg_emplvl,lq_total_annual_wages
0,"10 Total, all industries",9716618.0,141870066.0,1.0,1.0
6,"10 Total, all industries",9656240.0,139077079.0,1.0,1.0
4,"10 Total, all industries",9417676.0,120504622.0,1.0,1.0
26,102 Service-providing,8161795.0,99648903.0,1.0,1.0
30,"1021 Trade, transportation, and utilities",1916645.0,27002877.0,1.0,1.0
46,1025 Education and health services,1593362.0,21654265.0,1.0,1.0
5,"10 Total, all industries",298943.0,21365445.0,1.0,1.0
10,101 Goods-producing,1255881.0,20855718.0,1.0,1.0
42,1024 Professional and business services,1754985.0,20024917.0,1.0,1.0
3713,NAICS 62 Health care and social assistance,1479053.0,18887301.0,1.0,1.0


## Identify the NAICS Code by lengths 

In [17]:
blsqcew_titles.loc[:,'NAICS digits'] = 0
blsqcew_titles.loc[(blsqcew_titles['industry_code'].str.len()==2) |
    (blsqcew_titles['industry_code'].str.contains("-")),'NAICS digits'] = 2
blsqcew_titles.loc[(blsqcew_titles['industry_code'].str.len()==3),'NAICS digits'] = 3
blsqcew_titles.loc[(blsqcew_titles['industry_code'].str.len()==4),'NAICS digits'] = 4
blsqcew_titles.loc[(blsqcew_titles['industry_code'].str.len()==5),'NAICS digits'] = 5
blsqcew_titles.loc[(blsqcew_titles['industry_code'].str.len()==6),'NAICS digits'] = 6
blsqcew_titles.loc[(blsqcew_titles['industry_code'].str.startswith('10')),'NAICS digits'] = 0
blsqcew_titles.\
pivot_table(values='annual_avg_emplvl',index = 'NAICS digits',aggfunc=np.sum)

,annual_avg_emplvl
NAICS digits,
0,727922790.0
2,107909688.0
3,141851820.0
4,141842283.0
5,175620454.0
6,141608635.0


In [24]:
blsqcew_titles[['NAICS digits','industry_title']].loc[blsqcew_titles['NAICS digits']==2].head()

,NAICS digits,industry_title
62,2,"NAICS 11 Agriculture, forestry, fishing and hu..."
63,2,"NAICS 11 Agriculture, forestry, fishing and hu..."
64,2,"NAICS 11 Agriculture, forestry, fishing and hu..."
65,2,"NAICS 11 Agriculture, forestry, fishing and hu..."
286,2,"NAICS 21 Mining, quarrying, and oil and gas ex..."


In [21]:
#blsqcew_titles[['NAICS digits','industry_code']].tail(50)

Look at summary data by 2-digit NAICS code and Ownership Code.

In [25]:
blsqcew_2digit = blsqcew_titles[['industry_code','industry_title','own_code','annual_avg_estabs','annual_avg_emplvl']].\
loc[(blsqcew_titles['NAICS digits']==2)].sort_values(by=['industry_code','own_code'])
varformat = {('annual_avg_emplvl'): "{:,.0f}", ('annual_avg_estabs'):"{:,.0f}"}
blsqcew_2digit.head(8).style\
     .format(varformat)

,industry_code,industry_title,own_code,annual_avg_estabs,annual_avg_emplvl
62,11,"NAICS 11 Agriculture, forestry, fishing and hunting",1.000000,74,"3,174"
63,11,"NAICS 11 Agriculture, forestry, fishing and hunting",2.000000,143,"2,177"
64,11,"NAICS 11 Agriculture, forestry, fishing and hunting",3.000000,65,"1,132"
65,11,"NAICS 11 Agriculture, forestry, fishing and hunting",5.000000,"103,430","1,259,490"
286,21,"NAICS 21 Mining, quarrying, and oil and gas extraction",3.000000,10,243
287,21,"NAICS 21 Mining, quarrying, and oil and gas extraction",5.000000,"34,170","613,389"
354,22,NAICS 22 Utilities,1.000000,160,"12,182"
355,22,NAICS 22 Utilities,2.000000,69,0


Total jobs appears to be the toal of own code 9 (Total U.I. Covered (Excludes Federal Government)) and own code 1 (Federal Government). 

In [33]:
#  Need to drop total covered - own_code == 0 - leads to double counting
blsqcew_totalcovered = blsqcew_titles[['industry_code','industry_title','own_code','annual_avg_estabs','annual_avg_emplvl']].loc[(blsqcew_titles['own_code'] ==0)]
blsqcew_totalcovered.head()

,industry_code,industry_title,own_code,annual_avg_estabs,annual_avg_emplvl
0,10,"10 Total, all industries",0.0,9716618.0,141870066.0


In [28]:
# Replace Industry code for NAICS 92 to get levels of government
blsqcew_2digit.loc[(blsqcew_2digit['industry_code'] =='92')]

,industry_code,industry_title,own_code,annual_avg_estabs,annual_avg_emplvl
4597,92,NAICS 92 Public administration,1.0,24599.0,1518776.0
4598,92,NAICS 92 Public administration,2.0,48940.0,1787601.0
4599,92,NAICS 92 Public administration,3.0,61335.0,4001440.0


In [29]:
blsqcew_2digit.loc[(blsqcew_2digit['industry_code'] =='92') &
                   (blsqcew_2digit['own_code'] ==1),'industry_code'] = '92 1 Federal Government'
blsqcew_2digit.loc[(blsqcew_2digit['industry_code'] =='92') &
                   (blsqcew_2digit['own_code'] ==2),'industry_code'] = '92 2 State Government'
blsqcew_2digit.loc[(blsqcew_2digit['industry_code'] =='92') &
                   (blsqcew_2digit['own_code'] ==3),'industry_code'] = '92 3 Local Government'

In [30]:
table1 = blsqcew_2digit.groupby(by=["industry_code","industry_title"]).sum()
table1.reset_index(inplace = True)
table_title = "Two-digit NAICS, "+area_title
varformat = {('annual_avg_emplvl'): "{:,.0f}", ('annual_avg_estabs'):"{:,.0f}"}
table1.style\
     .set_caption(table_title)\
     .format(varformat)

,industry_code,industry_title,own_code,annual_avg_estabs,annual_avg_emplvl
0,11,"NAICS 11 Agriculture, forestry, fishing and hunting",11.000000,"103,712","1,265,973"
1,21,"NAICS 21 Mining, quarrying, and oil and gas extraction",8.000000,"34,180","613,632"
2,22,NAICS 22 Utilities,11.000000,"26,410","809,818"
3,23,NAICS 23 Construction,11.000000,"780,970","6,865,799"
4,42,NAICS 42 Wholesale trade,11.000000,"615,605","5,859,976"
5,51,NAICS 51 Information,11.000000,"162,474","2,940,560"
6,52,NAICS 52 Finance and insurance,11.000000,"481,611","5,860,656"
7,53,NAICS 53 Real estate and rental and leasing,11.000000,"378,339","2,170,326"
8,54,NAICS 54 Professional and technical services,11.000000,"1,167,987","8,946,511"
9,55,NAICS 55 Management of companies and enterprises,5.000000,"63,231","2,230,131"


# Create a function that obtains and cleans BLS data

In [71]:
import pandas as pd # For reading, writing and wrangling data
import sys  # saving CSV files

def obtain_clean_bls(area_fips: str = "US000", year: str = "2016"):
  """
  area_fips = 5 character string
  """
  blsqcew_areatitles = pd.read_csv('https://data.bls.gov/cew/doc/titles/area/area_titles.csv')
  blsqcew = pd.read_csv('http://data.bls.gov/cew/data/api/'+year+'/a/area/'+area_fips+'.csv')
  
  # Save Area Title for later use
  area_title_df = blsqcew_areatitles.loc[blsqcew_areatitles['area_fips'] == area_fips]
  area_title = area_title_df['area_title'].values[0]

  # Add industry titles
  industry_titles_df = pd.read_csv('https://www.bls.gov/cew/classifications/industry/industry-titles-csv.csv')
  blsqcew_titles = pd.merge(left = industry_titles_df,
                            right = blsqcew,
                            left_on = 'industry_code',
                            right_on = 'industry_code',
                            how = 'right')
  
  # Look at summary data by 2-digit NAICS code and Ownership Code
  blsqcew_2digit = blsqcew_titles[['industry_code','industry_title','own_code','annual_avg_estabs','annual_avg_emplvl']].\
  loc[(blsqcew_titles['industry_code'].str.len()==2) |
      (blsqcew_titles['industry_code'].str.contains("-"))].sort_values(by=['industry_code','own_code'])

  #  Need to drop total covered - own_code == 0, 8 and 9 - leads to double counting
  blsqcew_2digit = blsqcew_2digit.loc[(blsqcew['own_code'] !=0) |
                                      (blsqcew['own_code'] !=8) |
                                      (blsqcew['own_code'] !=9)]

  # Replace Industry code for NAICS 92 to get levels of government
  blsqcew_2digit.loc[(blsqcew_2digit['industry_code'] =='92') &
                   (blsqcew_2digit['own_code'] ==1),'industry_code'] = '92 1 Federal Government'
  blsqcew_2digit.loc[(blsqcew_2digit['industry_code'] =='92') &
                    (blsqcew_2digit['own_code'] ==2),'industry_code'] = '92 2 State Government'
  blsqcew_2digit.loc[(blsqcew_2digit['industry_code'] =='92') &
                    (blsqcew_2digit['own_code'] ==3),'industry_code'] = '92 3 Local Government'

  table1 = blsqcew_2digit.groupby(by=["industry_code","industry_title"]).sum()
  table1.reset_index(inplace = True)
  table_title = "Two-digit NAICS, "+area_title
  varformat = {('annual_avg_emplvl'): "{:,.0f}", ('annual_avg_estabs'):"{:,.0f}"}
  table1_fmt = table1.style\
      .set_caption(table_title)\
      .format(varformat)

  # renanme columns 

  # Save results as csv
  csv_filepath = 'BLSQCEW_'+year+"_"+area_fips+'.csv'
  savefile = csv_filepath
  print(savefile)
  table1.to_csv(savefile, index=False)

  return table1_fmt

obtain_clean_bls()

BLSQCEW_2016_US000.csv


,industry_code,industry_title,own_code,annual_avg_estabs,annual_avg_emplvl
0,10,"10 Total, all industries",28,"29,388,420","444,182,657"
1,11,"NAICS 11 Agriculture, forestry, fishing and hunting",11,"103,712","1,265,973"
2,21,"NAICS 21 Mining, quarrying, and oil and gas extraction",8,"34,180","613,632"
3,22,NAICS 22 Utilities,11,"26,410","809,818"
4,23,NAICS 23 Construction,11,"780,970","6,865,799"
5,31-33,NAICS 31-33 Manufacturing,11,"343,795","12,345,962"
6,42,NAICS 42 Wholesale trade,11,"615,605","5,859,976"
7,44-45,NAICS 44-45 Retail trade,11,"1,044,935","15,886,000"
8,48-49,NAICS 48-49 Transportation and warehousing,11,"272,822","5,723,644"
9,51,NAICS 51 Information,11,"162,474","2,940,560"
